In [4]:
import numpy as np
from scipy import optimize as opt

In [5]:
def qinteriorPoint(Q, A, b, c, guess, niter=20, tol=1e-16):
    
    n = len(c)  # length of x, c, μ
    m = len(b)  # length of b, y
    
    def F(x, y, μ):
        M = μ * np.identity(n)
        Y = y * np.identity(m)
        e = np.ones(n)
        top = np.array(Q @ x - A.T @ μ + c)
        middle = np.array(A @ x - y - b)
        bottom = np.array(Y @ M @ e)
        return np.concatenate((top, middle, bottom))
    
    σ = 1 / 10
    
    DF_top = np.hstack((Q, np.zeros((m, n)), -A.T))
    DF_middle = np.hstack((A, -np.identity(m), np.zeros((m, m))))
    
    def DF(M, Y):
        DF_bottom = np.hstack((np.zeros((m, n)), M, X))
        return np.vstack((DF_top, DF_middle, DF_bottom))
    
    it = 0
    dist = 100
    x, y, μ = guess
    while dist > tol and it < niter:
        ν = (y @ μ) / m
        e = np.ones(n)
        M = μ * np.identity(n)
        Y = y * np.identity(m)
        
        RHS = -F(x, y, μ) + np.concatenate((np.zeros(n + m), σ * ν * e))
        lu, piv = la.lu_factor(DF(M, Y))
        result = la.lu_solve((lu, piv), RHS)
        Δx = result[:n]
        Δy = result[n:m+n]
        Δμ = result[m+n:]

        τ = 0.95
        β_max = np.min([np.min(-μ[Δμ < 0] / Δμ[Δμ < 0]), 1.0])
        δ_max = np.min([np.min(-y[Δy < 0] / Δy[Δy < 0]), 1.0])

        β = np.min([1, τ * α_max])
        δ = np.min([1, τ * δ_max])
        α = np.min([β, δ])

        x_next = x + α * Δx
        y = y + α * Δy
        μ = μ + α * Δμ
        
        dist = la.norm(x_next - x)
        x = x_next
        it += 1
    return x, c @ x

In [6]:
def startingPoint(G, c, A, b, guess):
    """
    Obtain an appropriate initial point for solving the QP
    .5x\trp Gx+x\trp cs.t.Ax>=b. Parameters:
        G -- symmetric positive semidefinite matrix shape (n,n)
        c -- array of length n
        A -- constraint matrix shape (m,n)
        b -- array of length m
        guess -- a tuple of arrays (x, y, l) of lengths n, m, and m, resp.
    Returns:
        a tuple of arrays (x0, y0, l0) of lengths n, m, and m, resp.
    """
    m,n = A.shape
    x0, y0, l0 = guess
    # initialize linear system
    N = np.zeros((n+m+m, n+m+m))
    N[:n,:n] = G
    N[:n, n+m:] = -A.T
    N[n:n+m, :n] = A
    N[n:n+m, n:n+m] = -np.eye(m)
    N[n+m:, n:n+m] = np.diag(l0)
    N[n+m:, n+m:] = np.diag(y0)
    rhs = np.empty(n+m+m)
    rhs[:n] = -(G.dot(x0) - A.T.dot(l0)+c)
    rhs[n:n+m] = -(A.dot(x0) - y0 - b)
    rhs[n+m:] = -(y0*l0)
    sol = la.solve(N, rhs)
    dx = sol[:n]
    dy = sol[n:n+m]
    dl = sol[n+m:]
    y0 = np.maximum(1, np.abs(y0 + dy))
    l0 = np.maximum(1, np.abs(l0+dl))
    return x0, y0, l0
